In [56]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
import re;

In [55]:
def load_data():
    diagnosis = pd.read_csv('/Users/ravishchawla/Downloads/DIAGNOSES_ICD.csv');
    notes = pd.read_csv('/Users/ravishchawla/Downloads/NOTEEVENTS.csv');
    icd9 = pd.read_csv('/Users/ravishchawla/Downloads/ICD10_Formatted.csv.txt', sep='|', names=['ICD9_CODE', 'ICD10_CODE', 'desc']);
    
    return diagnosis, notes, icd9;

In [3]:
diagnosis, notes, icd9 = load_data();

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [248]:
notes_small = notes.ix[len(notes) - 20000:];
notes_small.shape

(20000, 11)

In [249]:
#groups = notes.groupby('HADM_ID').apply(lambda row: list(set(row['TEXT'])));

In [250]:
def get_stopwords():
    stop_words = str();
    with open('nltk', 'r') as f:
        for line in f:
            stop_words = stop_words + '\\b' + line.strip() + '\\b' + '|';

    stop_words = stop_words[0:-1];

    return stop_words;

In [427]:
def clean_text(notes_df):
    stop_words = get_stopwords();
    
    #notes_filtered = notes_df['TEXT'].apply(lambda row: re.sub("[^a-zA-Z0-9]", " ", row.lower()));
    notes_filtered = notes_df['TEXT'].apply(lambda row: re.sub("21[0-9]{2}.[0-1]?[0-9]{1}.[0-3]?[0-9]{1}.+[0-2]{1}[0-9]{1}:[0-5]{1}[0-9]{1}.+[\bAM\b|\bPM\b]", " ", row));
    notes_filtered = notes_filtered.apply(lambda row: re.sub("[^a-zA-Z0-9\.]", " ", row.lower()));
    #notes_filtered = notes_filtered.apply(lambda row: re.sub("\W\d+\.?\d*", "DIGIT", row));
    #notes_filtered = notes_filtered.apply(lambda row: re.sub("\s[a-zA-Z]\s", " ", row));                                        

    notes_nostops = notes_filtered.apply(lambda row: re.sub(stop_words, ' ', row));
    
    notes_final = notes_nostops.apply(lambda row: " ".join(row.split()));
    
    #notes_nonums = notes_final.apply(lambda)
    
    notes_df = notes_df.drop('TEXT', axis=1);
    notes_df = notes_df.assign(TEXT = notes_final.values)
    
    return notes_df;

In [428]:
notes_filtered = clean_text(notes_small);

In [429]:
notes_filtered.iloc[380]['TEXT']

'chief complaint ## hour events endoscopy sent sent ## ## escorted anesthesia. report called name## md ## rn holding area. received egd showed small gastric avm antral polyps clot ir consulted wouldn place g j given overlying left hepatic lobe bowel p trach g tube j tube doctor first name recs hold heparin gtt ## ## sqh ok feeds via j tube bowel sounds trach sutures ## ## days abd staples ## ## days allergies lorazepam confusion delir last dose antibiotics vancomycin infusions icu medications midazolam versed pantoprazole protonix dilantin heparin sodium prophylaxis fentanyl metoprolol medications changes medical family history review systems unchanged admission except noted review systems flowsheet data vital signs hemodynamic monitoring fluid balance ## hours since ## tmax ## c ## tcurrent ## c ## hr ## ## ## bpm bp ## ## ## ## ## ## ## ## ## mmhg rr ## ## ## insp min spo## ## heart rhythm sr sinus rhythm wgt current ## kg admission ## kg height ## inch total ## ## ml ## ## ml po tf 


    for row in ['aa bb cc dd ee ff', 'bb bb cc bb cc', 'aa aa', 'dd dd']:
        test = re.sub('(\\baa\\b|\\bbb\\b)', ' ', row);
        print(test)

In [383]:
vectorizer = CountVectorizer();

notes_counts = vectorizer.fit_transform(notes_filtered['TEXT']);

In [384]:
tfidf_transformer = TfidfTransformer();
tfidf_counts = tfidf_transformer.fit_transform(notes_counts);

In [385]:
zero_counts = np.where(tfidf_counts.toarray() == 0)[0].shape
print(zero_counts);

(594942971,)


In [386]:
tfidf_counts.shape[0] * tfidf_counts[1].shape[1] - zero_counts[0]

1737029

In [369]:
zero_counts = np.where(tfidf_counts.toarray() == 0)[0].shape
print(zero_counts);

(3148500,)


In [370]:
tfidf_counts.shape[0] * tfidf_counts[1].shape[1] - zero_counts[0]

11500

In [391]:
notes_filtered.iloc[0]['TEXT']

# Need to remove single charcater items
# Need to remove the leading 0 from a digit (i.e. 07 = 7) -OR- replace with "DIGIT"
# replace dates with "DATE"


'title chief complaint DIGITf pmh w chest pain found haveDIGITv cad proximal lad lcx total thrombotic occlusion p stenting DIGIT hour events cordis pa catheter placed wedge DIGIT diuresed withDIGITmg iv lasix x2 putting DIGITcc time. low dose metoprolol ordered given due borderline pressures iabp removed. allergies known drug allergies last dose antibiotics infusions heparin sodium DIGIT units hour icu medications furosemide lasix DIGITDIGITDIGIT DIGITDIGIT pm morphine sulfate DIGITDIGITDIGIT DIGITDIGIT pm medications changes medical family history review systems unchanged admission except noted review systems flowsheet data DIGITDIGITDIGIT DIGITDIGIT vital signs hemodynamic monitoring fluid balance DIGIT hours since DIGIT tmax DIGIT DIGIT tcurrent DIGIT DIGIT hr DIGIT DIGIT DIGIT bpm bp DIGITDIGITDIGIT DIGITDIGITDIGIT DIGITDIGITDIGIT mmhg rr DIGIT DIGIT DIGIT insp min spo2 DIGIT heart rhythm sr sinus rhythm height DIGIT inch cvp DIGIT DIGIT DIGIT mmhg pap DIGIT mmhg DIGIT mmhg co ci f

In [392]:
notes_small.iloc[0]['TEXT']

'TITLE:\n   Chief Complaint: 87F with no PMH, p/w chest pain, found to have 3V CAD\n   and proximal LAD and LCX total thrombotic occlusion, now s/p stenting\n   24 Hour Events:\n   - cordis, PA catheter placed with wedge -> 30\n   - diuresed with 40mg IV lasix x2, putting out ~1000cc each time.\n   - low dose metoprolol ordered, but not given due to borderline\n   pressures\n   - IABP removed.\n   Allergies:\n   No Known Drug Allergies\n   Last dose of Antibiotics:\n   Infusions:\n   Heparin Sodium - 750 units/hour\n   Other ICU medications:\n   Furosemide (Lasix) - [**2189-1-18**] 04:30 PM\n   Morphine Sulfate - [**2189-1-18**] 08:19 PM\n   Other medications:\n   Changes to medical and family history:\n   Review of systems is unchanged from admission except as noted below\n   Review of systems:\n   Flowsheet Data as of  [**2189-1-19**] 07:27 AM\n   Vital signs\n   Hemodynamic monitoring\n   Fluid balance\n                                                                  24 hours\n    